In [2]:
# Load the model and tokenizer from the saved directory
model = BertForSequenceClassification.from_pretrained(checkpoint_dir)
tokenizer = BertTokenizer.from_pretrained(checkpoint_dir)

def calculate_learning_measures(logins, time_spent, page_visits, search_queries, activity_completion, quiz_score, content_reactions_positive, content_reactions_negative, feedback):
    input_text = f"{logins} {time_spent} {page_visits} {search_queries} {activity_completion} {quiz_score} {content_reactions_positive} {content_reactions_negative} {feedback}"
    tokens = tokenizer(input_text, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
    tokens = {key: value.to('cuda' if torch.cuda.is_available() else 'cpu') for key, value in tokens.items()}
    with torch.no_grad():
        output = model(**tokens)
        scores = torch.sigmoid(output.logits).squeeze(0).cpu().numpy()

    return {
        'Conscientiousness': round(scores[0] * 10, 2),
        'Motivation': round(scores[1] * 10, 2),
        'Understanding': round(scores[2] * 10, 2),
        'Engagement': round(scores[3] * 10, 2)
    }

def generate_personalized_learning_report(learning_scores):
    """
    Generates a personalized learning report based on given learning scores.

    Args:
        learning_scores (dict): A dictionary containing learning measures scores.
            Expected keys: 'Conscientiousness', 'Motivation', 'Understanding', 'Engagement'.

    Returns:
        str: A formatted personalized learning report.
    """
    # Extracting scores
    conscientiousness = learning_scores.get('Conscientiousness', 0)
    motivation = learning_scores.get('Motivation', 0)
    understanding = learning_scores.get('Understanding', 0)
    engagement = learning_scores.get('Engagement', 0)

    # Report Header
    report = [
        "=== Personalized Learning Report ===",
        f"Conscientiousness: {conscientiousness}/10",
        f"Motivation: {motivation}/10",
        f"Understanding: {understanding}/10",
        f"Engagement: {engagement}/10",
        ""
    ]

    return "\n".join(report)

personalized_report = generate_personalized_learning_report(learning_scores)
print(personalized_report)

=== Personalized Learning Report ===
Conscientiousness: 6.49/10
Motivation: 6.64/10
Understanding: 5.35/10
Engagement: 5.89/10



In [ ]:
!pip install gradientai --upgrade
from gradientai import Gradient
import os

os.environ['GRADIENT_ACCESS_TOKEN'] = ''
os.environ['GRADIENT_WORKSPACE_ID'] = ''

# Initialize the Gradient SDK
gradient = Gradient()

# Get the base model and create a new model adapter
base_model = gradient.get_base_model(base_model_slug="llama2-7b-chat")
new_model_adapter = base_model.create_model_adapter(name="test model 3")
print(f"Created model adapter with id {new_model_adapter.id}")

class ContextualChatbot:
    def __init__(self, model_adapter, learning_scores):
        self.model_adapter = model_adapter
        self.learning_scores = learning_scores
        self.conversation_history = ""

    def generate_response(self, user_prompt):
        """
        Generates a response based on user input and existing conversation history.

        Args:
            user_prompt (str): The input question or prompt provided by the user.

        Returns:
            str: A contextual response generated by the model adapter.
        """
        # Update the conversation history with the latest user prompt
        self.conversation_history += f"\nUser: {user_prompt}\n"

        # Create personalization context
        personalization_context = (f"As an assistant, I understand your learning characteristics as follows:\n"
                                   f"Conscientiousness: {self.learning_scores['Conscientiousness']}, "
                                   f"Motivation: {self.learning_scores['Motivation']}, "
                                   f"Understanding: {self.learning_scores['Understanding']}, "
                                   f"Engagement: {self.learning_scores['Engagement']}.\n"
                                   "Let's continue our chat.\n")

        # Create the complete prompt by combining the personalization context and conversation history
        complete_prompt = personalization_context + self.conversation_history

        # Generate a response using the model adapter
        response = new_model_adapter.complete(query=user_prompt, max_generated_token_count=511).generated_output

        # Update the conversation history with the assistant's response
        self.conversation_history += f"Assistant: {response}\n"

        return response

# Example learning scores (replace with real data)
learning_scores = {
    'Conscientiousness': 8.0,
    'Motivation': 7.5,
    'Understanding': 9.0,
    'Engagement': 7.0
}

# Initialize the chatbot instance
chatbot = ContextualChatbot(model_adapter=new_model_adapter, learning_scores=learning_scores)

# Continuously chat with the user until they decide to quit
print("Welcome to the personalized chatbot! Type 'quit' to exit.")
while True:
    user_prompt = input("\nPlease enter your query or question: ")

    if user_prompt.lower() in ["quit", "exit", "q"]:
        print("Exiting the chat. Goodbye!")
        break

    response = chatbot.generate_response(user_prompt)
    print("\nPersonalized Response:", response)


Created model adapter with id e84d6e4c-e6b4-4ff2-a1cd-2c364f287a0d_model_adapter
Welcome to the personalized chatbot! Type 'quit' to exit.

Please enter your query or question: How to improve my reading skills

Personalized Response:  Improving your reading skills can be a challenging task, but with consistent practice and dedication, you can achieve significant progress. Here are some effective ways to improve your reading skills:

1. Set specific goals: Setting specific goals for your reading will help you focus your efforts and track your progress. Identify areas you want to improve, such as comprehension, vocabulary, or speed, and set achievable goals for yourself.
2. Practice regularly: Regular practice is essential to improve your reading skills. Allocate time each day to read, whether it's a book, article, or newsletter. Consistency is key to developing muscle memory and improving your reading abilities.
3. Diversify your reading material: Expose yourself to different types of t